In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import gc
import pickle
import joblib
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

### Загрузка данных

In [25]:
# Загрузка данных
print("Загрузка данных...")
train = pd.read_csv('./data/train_raw.csv')
val = pd.read_csv('./data/val_raw.csv')
test = pd.read_csv('./data/test_raw.csv')

print(f"Размеры датасетов:")
print(f"Train: {train.shape}")
print(f"Val: {val.shape}")
print(f"Test: {test.shape}")
print(f"\nКолонок: {train.shape[1]}")
print(f"ID колонка: {'SK_ID_CURR' in train.columns}")
print(f"Target колонка: {'TARGET' in train.columns}")

# Проверяем распределение таргета
if 'TARGET' in train.columns:
    target_dist = train['TARGET'].value_counts(normalize=True)
    print(f"\nРаспределение TARGET в train:")
    print(f"0: {target_dist[0]:.2%} ({train['TARGET'].value_counts()[0]} записей)")
    print(f"1: {target_dist[1]:.2%} ({train['TARGET'].value_counts()[1]} записей)")

Загрузка данных...
Размеры датасетов:
Train: (184506, 1337)
Val: (61502, 1337)
Test: (61503, 1337)

Колонок: 1337
ID колонка: True
Target колонка: True

Распределение TARGET в train:
0: 91.93% (169611 записей)
1: 8.07% (14895 записей)


### Отделяем таргет от признаков

In [26]:
# Сохраняем ID и target отдельно
train_ids = train['SK_ID_CURR']
val_ids = val['SK_ID_CURR']
test_ids = test['SK_ID_CURR']

train_target = train['TARGET']
val_target = val['TARGET']
test_target = test['TARGET']

# Удаляем из features
X_train = train.drop(columns=['SK_ID_CURR', 'TARGET'])
X_val = val.drop(columns=['SK_ID_CURR', 'TARGET'])
X_test = test.drop(columns=['SK_ID_CURR', 'TARGET'])

print(f"Features shape: {X_train.shape}")
print(f"Train target shape: {train_target.shape}")

del train, val, test
gc.collect()

Features shape: (184506, 1335)
Train target shape: (184506,)


0

## Удаление столбцов с большим числом пропусков

In [27]:
def remove_high_missing_cols(X_train, X_val, X_test, missing_threshold=0.5):
    """Удаляет колонки с большим процентом пропусков (на основе анализа train)"""
    # Анализируем пропуски только в train
    missing_ratios = X_train.isnull().mean()
    high_missing_cols = missing_ratios[missing_ratios > missing_threshold].index.tolist()
    
    print(f"Колонок с >{missing_threshold*100}% пропусков в train: {len(high_missing_cols)}")
    
    if high_missing_cols:
        print("\nТоп-10 колонок с наибольшим процентом пропусков:")
        for col, perc in missing_ratios[high_missing_cols].sort_values(ascending=False).head(10).items():
            print(f"  {col}: {perc*100:.1f}%")
    
    # Удаляем из всех датасетов
    X_train_clean = X_train.drop(columns=high_missing_cols)
    X_val_clean = X_val.drop(columns=[col for col in high_missing_cols if col in X_val.columns])
    X_test_clean = X_test.drop(columns=[col for col in high_missing_cols if col in X_test.columns])
    
    print(f"\nРазмеры после удаления:")
    print(f"Train: {X_train_clean.shape}")
    print(f"Val: {X_val_clean.shape}")
    print(f"Test: {X_test_clean.shape}")
    
    
    return X_train_clean, X_val_clean, X_test_clean, high_missing_cols

# Применяем функцию
X_train, X_val, X_test, high_missing_cols = remove_high_missing_cols(
    X_train, X_val, X_test, missing_threshold=0.5
)

gc.collect()

Колонок с >50.0% пропусков в train: 483

Топ-10 колонок с наибольшим процентом пропусков:
  client_credit_AMT_PAYMENT_CURRENT_min_min: 80.1%
  client_credit_AMT_PAYMENT_CURRENT_min_max: 80.1%
  client_credit_AMT_PAYMENT_CURRENT_min_mean: 80.1%
  client_credit_AMT_PAYMENT_CURRENT_mean_mean: 80.1%
  client_credit_AMT_PAYMENT_CURRENT_mean_min: 80.1%
  client_credit_AMT_PAYMENT_CURRENT_mean_max: 80.1%
  client_credit_AMT_PAYMENT_CURRENT_max_mean: 80.1%
  client_credit_AMT_PAYMENT_CURRENT_max_min: 80.1%
  client_credit_AMT_PAYMENT_CURRENT_max_max: 80.1%
  client_credit_CNT_DRAWINGS_POS_CURRENT_max_mean: 80.1%

Размеры после удаления:
Train: (184506, 852)
Val: (61502, 852)
Test: (61503, 852)


0

## Анализ и обработка выбросов

In [28]:
# 1. Обработка DAYS_EMPLOYED (специфичная аномалия)
print("1. Обработка DAYS_EMPLOYED...")
days_employed_col = [col for col in X_train.columns if 'DAYS_EMPLOYED' in col]

if days_employed_col:
    col_name = days_employed_col[0]
    
    # Создаем признак аномалии
    X_train[f'{col_name}_ANOM'] = (X_train[col_name] == 365243).astype(int)
    X_val[f'{col_name}_ANOM'] = (X_val[col_name] == 365243).astype(int)
    X_test[f'{col_name}_ANOM'] = (X_test[col_name] == 365243).astype(int)
    
    # Заменяем 365243 на NaN
    X_train[col_name] = X_train[col_name].replace({365243: np.nan})
    X_val[col_name] = X_val[col_name].replace({365243: np.nan})
    X_test[col_name] = X_test[col_name].replace({365243: np.nan})
    
    print(f"   Создан признак {col_name}_ANOM")
    print(f"   Заменено {X_train[f'{col_name}_ANOM'].sum()} значений 365243 на NaN в train")

# 2. Находим все признаки с выбросами
print("\n2. Поиск признаков с выбросами...")
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

outliers_list = []
for col in numeric_cols:
    data = X_train[col].dropna()
    if len(data) > 10:
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        
        if IQR > 0:
            lower = Q1 - 3 * IQR
            upper = Q3 + 3 * IQR
            
            outliers = ((data < lower) | (data > upper)).sum()
            if outliers > 0:
                outlier_perc = outliers / len(data) * 100
                outliers_list.append((col, outliers, outlier_perc))

print(f"   Найдено признаков с выбросами: {len(outliers_list)}")

# 3. Сортируем по проценту выбросов и показываем топ
if outliers_list:
    outliers_df = pd.DataFrame(outliers_list, columns=['feature', 'n_outliers', 'outlier_perc'])
    outliers_df = outliers_df.sort_values('outlier_perc', ascending=False)
    
    print("\n   Топ-15 признаков с выбросами:")
    print(outliers_df.head(15).to_string())
    
    # 4. Обрабатываем выбросы - простой метод winsorization (обрезаем)
    print("\n3. Обработка выбросов (winsorization 1%-99%)...")
    
    for col in outliers_df['feature']:
        if col in X_train.columns:
            # Берем 1% и 99% перцентили из train
            lower = X_train[col].quantile(0.01)
            upper = X_train[col].quantile(0.99)
            
            # Обрабатываем все датасеты
            X_train[col] = X_train[col].clip(lower, upper)
            if col in X_val.columns:
                X_val[col] = X_val[col].clip(lower, upper)
            if col in X_test.columns:
                X_test[col] = X_test[col].clip(lower, upper)
    
    print(f"   Обработано признаков: {len(outliers_df)}")

print("\nРазмеры после обработки выбросов:")
print(f"Train: {X_train.shape}")
print(f"Val: {X_val.shape}")
print(f"Test: {X_test.shape}")

gc.collect()

1. Обработка DAYS_EMPLOYED...
   Создан признак DAYS_EMPLOYED_ANOM
   Заменено 33122 значений 365243 на NaN в train

2. Поиск признаков с выбросами...
   Найдено признаков с выбросами: 359

   Топ-15 признаков с выбросами:
                                                  feature  n_outliers  outlier_perc
293  client_installments_NUM_INSTALMENT_VERSION_mean_mean       42228     24.130286
40                      bureau_AMT_CREDIT_MAX_OVERDUE_sum       37916     23.972434
87               client_bureau_balance_STATUS_X_count_sum       31967     20.211172
89          client_bureau_balance_STATUS_X_count_norm_sum       31608     19.984194
83               client_bureau_balance_STATUS_C_count_sum       27747     17.543072
29                         bureau_DAYS_CREDIT_ENDDATE_max       26867     17.136315
55                          bureau_DAYS_CREDIT_UPDATE_max       26688     16.873518
91                previous_NAME_PRODUCT_TYPE_walk-in_mean       29310     16.783481
151                  

0

## Заполнение пропущенных значений

In [29]:
def impute_missing_no_leakage(X_train, X_val, X_test):
    """Заполняет пропуски без утечек данных (все значения из train)"""
    print("Заполнение пропущенных значений...")
    
    # Создаем копии
    X_train_imputed = X_train.copy()
    X_val_imputed = X_val.copy()
    X_test_imputed = X_test.copy()
    
    # Сохраняем значения для импутации
    imputation_values = {
        'numeric': {},
        'categorical': {}
    }
    
    # 1. Числовые колонки - заполняем медианой из train
    numeric_cols = X_train_imputed.select_dtypes(include=[np.number]).columns.tolist()
    
    for col in numeric_cols:
        if X_train_imputed[col].isnull().any():
            # Вычисляем медиану на train
            fill_value = X_train_imputed[col].median()
            imputation_values['numeric'][col] = fill_value
            
            # Заполняем train
            X_train_imputed[col] = X_train_imputed[col].fillna(fill_value)
            
            # Заполняем val и test теми же значениями
            if col in X_val_imputed.columns:
                X_val_imputed[col] = X_val_imputed[col].fillna(fill_value)
            if col in X_test_imputed.columns:
                X_test_imputed[col] = X_test_imputed[col].fillna(fill_value)
    
    # 2. Категориальные колонки - заполняем 'MISSING'
    categorical_cols = X_train_imputed.select_dtypes(include=['object']).columns.tolist()
    
    for col in categorical_cols:
        if X_train_imputed[col].isnull().any():
            fill_value = 'MISSING'
            imputation_values['categorical'][col] = fill_value
            
            # Заполняем train
            X_train_imputed[col] = X_train_imputed[col].fillna(fill_value)
            
            # Заполняем val и test теми же значениями
            if col in X_val_imputed.columns:
                X_val_imputed[col] = X_val_imputed[col].fillna(fill_value)
            if col in X_test_imputed.columns:
                X_test_imputed[col] = X_test_imputed[col].fillna(fill_value)
    
    # Проверяем результат
    print(f"\nРезультат импутации:")
    print(f"Заполнено числовых колонок: {len(imputation_values['numeric'])}")
    print(f"Заполнено категориальных колонок: {len(imputation_values['categorical'])}")
    print(f"\nПропусков после импутации:")
    print(f"Train: {X_train_imputed.isnull().sum().sum()}")
    print(f"Val: {X_val_imputed.isnull().sum().sum()}")
    print(f"Test: {X_test_imputed.isnull().sum().sum()}")
    
    
    return X_train_imputed, X_val_imputed, X_test_imputed, imputation_values

# Заполняем пропуски
X_train, X_val, X_test, imputation_values = impute_missing_no_leakage(X_train, X_val, X_test)

gc.collect()

Заполнение пропущенных значений...

Результат импутации:
Заполнено числовых колонок: 796
Заполнено категориальных колонок: 3

Пропусков после импутации:
Train: 0
Val: 0
Test: 0


0

## Обработка категорильных признаков

In [30]:
def encode_categorical_no_leakage(X_train, X_val, X_test, max_categories_ohe=20):
    """Кодирует категориальные признаки без утечек данных"""
    # Определяем категориальные колонки
    categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
    
    if not categorical_cols:
        print("Нет категориальных признаков для кодирования")
        return X_train, X_val, X_test, {}
    
    print(f"Найдено {len(categorical_cols)} категориальных признаков")
    
    # Разделяем на low и high cardinality
    low_cardinality = []
    high_cardinality = []
    
    for col in categorical_cols:
        n_unique = X_train[col].nunique()
        if n_unique <= max_categories_ohe:
            low_cardinality.append(col)
        else:
            high_cardinality.append(col)
    
    print(f"One-Hot Encoding для {len(low_cardinality)} признаков (≤{max_categories_ohe} уникальных)")
    print(f"Label Encoding для {len(high_cardinality)} признаков (>{max_categories_ohe} уникальных)")
    
    # Словарь для хранения энкодеров
    encoders = {
        'label_encoders': {},
        'onehot_encoder': None,
        'low_cardinality': low_cardinality,
        'high_cardinality': high_cardinality
    }
    
    # 1. Label Encoding для high cardinality
    if high_cardinality:
        print("\nПрименение Label Encoding...")
        for col in tqdm(high_cardinality, desc="Label Encoding"):
            # Обучаем на train
            le = LabelEncoder()
            le.fit(X_train[col].astype(str))
            encoders['label_encoders'][col] = le
            
            # Применяем к train
            X_train[col] = le.transform(X_train[col].astype(str))
            
            # Применяем к val (неизвестные -> -1)
            if col in X_val.columns:
                X_val[col] = X_val[col].astype(str).apply(
                    lambda x: le.transform([x])[0] if x in le.classes_ else -1
                )
            
            # Применяем к test (неизвестные -> -1)
            if col in X_test.columns:
                X_test[col] = X_test[col].astype(str).apply(
                    lambda x: le.transform([x])[0] if x in le.classes_ else -1
                )
    
    # 2. One-Hot Encoding для low cardinality
    if low_cardinality:
        print("\nПрименение One-Hot Encoding...")
        
        # Обучаем на train
        ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=np.int8)
        ohe.fit(X_train[low_cardinality].astype(str))
        encoders['onehot_encoder'] = ohe
        
        # Применяем ко всем датасетам
        X_train_ohe = ohe.transform(X_train[low_cardinality].astype(str))
        X_val_ohe = ohe.transform(X_val[low_cardinality].astype(str))
        X_test_ohe = ohe.transform(X_test[low_cardinality].astype(str))
        
        # Создаем датафреймы
        ohe_columns = ohe.get_feature_names_out(low_cardinality)
        
        X_train_ohe_df = pd.DataFrame(X_train_ohe, columns=ohe_columns, index=X_train.index)
        X_val_ohe_df = pd.DataFrame(X_val_ohe, columns=ohe_columns, index=X_val.index)
        X_test_ohe_df = pd.DataFrame(X_test_ohe, columns=ohe_columns, index=X_test.index)
        
        # Удаляем исходные колонки и добавляем OHE
        X_train = X_train.drop(columns=low_cardinality)
        X_val = X_val.drop(columns=low_cardinality)
        X_test = X_test.drop(columns=low_cardinality)
        
        X_train = pd.concat([X_train, X_train_ohe_df], axis=1)
        X_val = pd.concat([X_val, X_val_ohe_df], axis=1)
        X_test = pd.concat([X_test, X_test_ohe_df], axis=1)
    
    print(f"\nРазмеры после кодирования:")
    print(f"Train: {X_train.shape}")
    print(f"Val: {X_val.shape}")
    print(f"Test: {X_test.shape}")
    
    
    return X_train, X_val, X_test, encoders

# Кодируем категориальные признаки
X_train, X_val, X_test, categorical_encoders = encode_categorical_no_leakage(
    X_train, X_val, X_test, max_categories_ohe=20
)

gc.collect()

Найдено 13 категориальных признаков
One-Hot Encoding для 12 признаков (≤20 уникальных)
Label Encoding для 1 признаков (>20 уникальных)

Применение Label Encoding...


Label Encoding: 100%|██████████| 1/1 [00:07<00:00,  7.13s/it]



Применение One-Hot Encoding...

Размеры после кодирования:
Train: (184506, 912)
Val: (61502, 912)
Test: (61503, 912)


0

## Удаление сильно коррелированных признаков

In [31]:
def remove_correlated_features_no_leakage(X_train, X_val, X_test, correlation_threshold=0.95):
    """Удаляет сильно коррелированные признаки (анализ только на train)"""
    print(f"Поиск коррелированных признаков (порог: {correlation_threshold})...")
    
    # Вычисляем корреляционную матрицу на train
    corr_matrix = X_train.corr().abs()
    
    # Верхний треугольник матрицы
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Находим колонки для удаления
    to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]
    
    print(f"Найдено коррелированных признаков для удаления: {len(to_drop)}")
    
    if to_drop:
        print("\nПримеры удаляемых признаков:")
        for col in to_drop[:10]:
            # Найдем наиболее коррелированный признак
            corr_values = upper[col]
            if not corr_values.empty:
                max_corr = corr_values.max()
                corr_with = corr_values[corr_values == max_corr].index
                if len(corr_with) > 0:
                    print(f"  {col} (корреляция {max_corr:.3f} с {list(corr_with)[0]})")
    
    # Удаляем колонки из всех датасетов
    X_train_uncorr = X_train.drop(columns=to_drop)
    X_val_uncorr = X_val.drop(columns=[col for col in to_drop if col in X_val.columns])
    X_test_uncorr = X_test.drop(columns=[col for col in to_drop if col in X_test.columns])
    
    print(f"\nРазмеры после удаления:")
    print(f"Train: {X_train_uncorr.shape}")
    print(f"Val: {X_val_uncorr.shape}")
    print(f"Test: {X_test_uncorr.shape}")
    
    
    return X_train_uncorr, X_val_uncorr, X_test_uncorr, to_drop

# Удаляем коррелированные признаки
X_train, X_val, X_test, correlated_cols = remove_correlated_features_no_leakage(
    X_train, X_val, X_test, correlation_threshold=0.95
)

gc.collect()

Поиск коррелированных признаков (порог: 0.95)...
Найдено коррелированных признаков для удаления: 211

Примеры удаляемых признаков:
  AMT_GOODS_PRICE (корреляция 0.986 с AMT_CREDIT)
  REGION_RATING_CLIENT_W_CITY (корреляция 0.951 с REGION_RATING_CLIENT)
  YEARS_BEGINEXPLUATATION_MODE (корреляция 0.986 с YEARS_BEGINEXPLUATATION_AVG)
  FLOORSMAX_MODE (корреляция 0.987 с FLOORSMAX_AVG)
  YEARS_BEGINEXPLUATATION_MEDI (корреляция 0.996 с YEARS_BEGINEXPLUATATION_AVG)
  FLOORSMAX_MEDI (корреляция 0.997 с FLOORSMAX_AVG)
  OBS_60_CNT_SOCIAL_CIRCLE (корреляция 0.998 с OBS_30_CNT_SOCIAL_CIRCLE)
  bureau_CREDIT_ACTIVE_Closed_count_norm (корреляция 0.992 с bureau_CREDIT_ACTIVE_Active_count_norm)
  bureau_CREDIT_CURRENCY_currency 2_count_norm (корреляция 0.974 с bureau_CREDIT_CURRENCY_currency 1_count_norm)
  bureau_CREDIT_TYPE_Interbank credit_count_norm (корреляция 1.000 с bureau_CREDIT_TYPE_Interbank credit_count)

Размеры после удаления:
Train: (184506, 701)
Val: (61502, 701)
Test: (61503, 701)


0

In [32]:
def save_processed_data(X_train, X_val, X_test, train_ids, val_ids, test_ids, 
                       train_target, val_target, test_target):
    """Сохраняет обработанные данные"""
    print("Сохранение обработанных данных...")
    
    # 1. Проверяем совпадение колонок
    train_cols = set(X_train.columns)
    val_cols = set(X_val.columns)
    test_cols = set(X_test.columns)
    
    print(f"\n1. СОВПАДЕНИЕ КОЛОНОК:")
    print(f"   Train колонок: {len(train_cols)}")
    print(f"   Val колонок: {len(val_cols)}")
    print(f"   Test колонок: {len(test_cols)}")
    print(f"   Совпадение train и val: {train_cols == val_cols}")
    print(f"   Совпадение train и test: {train_cols == test_cols}")
    
    # 2. Проверяем пропуски
    print(f"\n2. ПРОВЕРКА ПРОПУСКОВ:")
    print(f"   Train пропусков: {X_train.isnull().sum().sum()}")
    print(f"   Val пропусков: {X_val.isnull().sum().sum()}")
    print(f"   Test пропусков: {X_test.isnull().sum().sum()}")
    
    # 3. Проверяем типы данных
    print(f"\n3. ТИПЫ ДАННЫХ:")
    dtype_counts = X_train.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"   {dtype}: {count} колонок")
    
    # 4. Собираем финальные датасеты
    print(f"\n4. СОХРАНЕНИЕ ДАННЫХ...")
    
    # Собираем полные датасеты
    train_final = pd.concat([train_ids.reset_index(drop=True), 
                             train_target.reset_index(drop=True), 
                             X_train.reset_index(drop=True)], axis=1)
    
    val_final = pd.concat([val_ids.reset_index(drop=True), 
                           val_target.reset_index(drop=True), 
                           X_val.reset_index(drop=True)], axis=1)
    
    test_final = pd.concat([test_ids.reset_index(drop=True), 
                            test_target.reset_index(drop=True), 
                            X_test.reset_index(drop=True)], axis=1)
    
    # Сохраняем полные датасеты
    train_final.to_csv('./data/train_processed.csv', index=False)
    val_final.to_csv('./data/val_processed.csv', index=False)
    test_final.to_csv('./data/test_processed.csv', index=False)
    
    print(f"   train_processed.csv: {train_final.shape}")
    print(f"   val_processed.csv: {val_final.shape}")
    print(f"   test_processed.csv: {test_final.shape}")
    
    
    return train_final, val_final, test_final

# Сохраняем данные
train_final, val_final, test_final = save_processed_data(
    X_train, X_val, X_test,
    train_ids, val_ids, test_ids,
    train_target, val_target, test_target
)

print(f"\nИтоговые размеры:")
print(f"Train: {train_final.shape}")
print(f"Val: {val_final.shape}")
print(f"Test: {test_final.shape}")
print(f"\nВсего признаков: {X_train.shape[1]}")
print(f"Распределение таргета в train: {train_target.mean():.4f}")

Сохранение обработанных данных...

1. СОВПАДЕНИЕ КОЛОНОК:
   Train колонок: 701
   Val колонок: 701
   Test колонок: 701
   Совпадение train и val: True
   Совпадение train и test: True

2. ПРОВЕРКА ПРОПУСКОВ:
   Train пропусков: 0
   Val пропусков: 0
   Test пропусков: 0

3. ТИПЫ ДАННЫХ:
   float64: 597 колонок
   int8: 65 колонок
   int64: 39 колонок

4. СОХРАНЕНИЕ ДАННЫХ...
   train_processed.csv: (184506, 703)
   val_processed.csv: (61502, 703)
   test_processed.csv: (61503, 703)

Итоговые размеры:
Train: (184506, 703)
Val: (61502, 703)
Test: (61503, 703)

Всего признаков: 701
Распределение таргета в train: 0.0807


In [33]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184506 entries, 0 to 184505
Columns: 703 entries, SK_ID_CURR to EMERGENCYSTATE_MODE_Yes
dtypes: float64(597), int64(41), int8(65)
memory usage: 909.5 MB


# Вывод

Данные были обработаны. Удалены призанки с большим количеством пропусков. Обработаны выбросы. ЗАполнены пропущенные значения. Удалены сильно коррелирвоанные признаки. 

Готовые обработанные датасеты сохранены в файлы.